<a href="https://colab.research.google.com/github/cssaivishnu/IR_Repo/blob/main/classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import the essential libraries and mount the drive

In [ ]:
!pip install tensorflow-addons

In [ ]:
import numpy as np
import os
import json
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow import keras
from tensorflow.keras import layers

np.random.seed = 20
tf.random.set_seed(20)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Make the main repo as the current active repository

In [ ]:
os.chdir('/content/drive/MyDrive/IR_repo')
cwd = os.getcwd()
print(cwd)

/content/drive/MyDrive/IR_repo


Initialisation

In [ ]:
input_shape = (56, 56, 3)
names_list = os.listdir('dataset')
num_categories = len(names_list)

Load the name_to_label and label_to_name encodings

In [ ]:
with open('name_to_label.json', 'r') as f:
    name_to_label = json.load(f)
with open('label_to_name.json', 'r') as f:
    label_to_name = json.load(f)

Load the features(X) and labels(y) from their respective numpy files

In [ ]:
X = np.load('features.npy')
y = np.load('labels.npy')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
print('X_train:', X_train.shape)
print('X_test:', X_test.shape)
print('y_train:', y_train.shape)
print('y_test:', y_test.shape)

X_train: (8800, 56, 56, 3)
X_test: (2200, 56, 56, 3)
y_train: (8800, 1)
y_test: (2200, 1)


Image Data Augmentation

In [ ]:
data_augmentation = keras.Sequential(
    [
        layers.Normalization(),
        layers.RandomFlip('horizontal'),
        layers.RandomRotation(0.02),
        layers.RandomWidth(0.2),
        layers.RandomHeight(0.2)
    ]
)

data_augmentation.layers[0].adapt(X_train)

Define the Supervised Contrastive Loss Function that will be used in training the model 

In [ ]:
class SupervisedContrastiveLoss(keras.losses.Loss):
    def __init__(self, temperature=1, name=None):
        super(SupervisedContrastiveLoss, self).__init__(name=name)
        self.temperature = temperature
    def __call__(self, labels, feature_vectors, sample_weight=None):
        # normalize the feature vectors
        feature_vectors_normailzed = tf.math.l2_normalize(feature_vectors, axis=1)
        # Compute Logits
        logits = tf.divide(
            tf.matmul(
                feature_vectors_normailzed, tf.transpose(feature_vectors_normailzed)
            ),
            self.temperature,
        )
        return tfa.losses.npairs_loss(tf.squeeze(labels), logits)

Create an encoder to encode the images pixel data using the RNN - ResNet50V2

In [ ]:
def create_encoder():
    resnet = keras.applications.ResNet50V2(
        include_top=False, weights=None, input_shape=input_shape, pooling='avg'
    )
    inputs = keras.Input(shape=input_shape)
    augmented = data_augmentation(inputs)
    outputs = resnet(augmented)
    model = keras.Model(inputs=inputs, outputs=outputs, name='encoder')
    return model

Adding a Projection head

In [ ]:
def add_projection_head(encoder, projection_units):
    inputs = keras.Input(shape=input_shape)
    features = encoder(inputs)
    outputs = layers.Dense(projection_units, activation='relu')(features)
    model = keras.Model(
        inputs=inputs, outputs=outputs, name='encoder_with_projection_head'
    )
    return model

Train the encoder with Supervised Contrastive Loss defined above for better encoding

In [ ]:
learning_rate = 0.001
batch_size = 100
projection_units = 256
epochs = 50
temperature = 0.05

encoder = create_encoder()
encoder.summary()
encoder_with_projection_head = add_projection_head(encoder, projection_units)
encoder_with_projection_head.compile(
    optimizer=keras.optimizers.Adam(learning_rate),
    loss=SupervisedContrastiveLoss(temperature)
)
encoder_with_projection_head.summary()

with tf.device('/gpu:0'):
    history = encoder_with_projection_head.fit(
        x=X_train, y=y_train, batch_size=batch_size, epochs=epochs
    )

Creating and training the classifier using the pretrained encoder

In [ ]:
def create_classifier(encoder, dropout_rate, hidden_units, learning_rate, trainable=True):
    for layer in encoder.layers:
        layer.trainable = trainable
    inputs = keras.Input(shape=input_shape)
    features = encoder(inputs)
    features = layers.Dropout(dropout_rate)(features)
    features = layers.Dense(hidden_units, activation='relu')(features)
    features = layers.Dropout(dropout_rate)(features)
    outputs = layers.Dense(num_categories, activation='softmax')(features)

    model = keras.Model(inputs=inputs, outputs=outputs, name = 'classifier')
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate),
        loss=keras.losses.SparseCategoricalCrossentropy(),
        metrics=[keras.metrics.SparseCategoricalAccuracy()],
    )
    return model

In [ ]:
dropout_rate = 0.5
hidden_units = 512
learning_rate = 0.001
batch_size = 100
epochs = 100

classifier = create_classifier(encoder, dropout_rate, hidden_units, learning_rate, trainable=False)

with tf.device('/gpu:0'):
    history = classifier.fit(x=X_train, y=y_train, batch_size=batch_size, epochs=epochs, validation_split=0.2, stratify=y_train)

accuracy = classifier.evaluate(X_test, y_test)[1]
print(f'Test Accuracy: {round(accuracy*100,2)}%')

In [ ]:
classifier.save('classifier.h5')